In [1]:
%run generate_new_examples.ipynb     # importing functions

# Closest and minimal vectors

The idea is, to compare the following vectors:

`closest`: the vector of the cube, which is *closest* to the vector given by the LLL algorithm

`minimal`: vector of the cube, whichever of them has the smallest norm

We would expect, that the closest and minimal vectors would be the same, meaning that the norm function decreases the most in the direction of the closest vector.




Another way would be comparing the gradient.


<details>
  <summary>Click to expand!</summary>
  
  ## Heading
  1. A numbered
  2. list
     * With some
     * Sub bullets
</details>


In [2]:
### closest_versus_minimal

def closest_versus_grammatrix(jsonfilename):
    cases = from_json(jsonfilename)
    for case in cases:
        closest = closest_point_in_cube(case["lincomb_cube"], case["lincomb_LLL"])
        point_versus_grammatrix(case["G"], case["lincomb_LLL"], closest)
    
def point_versus_grammatrix(G: list, lc_LLL: list, point: list) -> None:
    detailed_product, sum = matrix_multiplication_detailed(matrix(G), vector(point)-vector(lc_LLL))
    print("lc in question: {}, lc LLL: {}". format(point, lc_LLL))
    print("difference between the two lcs/points:", vector(point)-vector(lc_LLL))
    print(detailed_product)
    print(sum)
    print()
        

In [3]:
# closest_versus_grammatrix("matrices3x3-600instances.json")

In [4]:
def cube_versus_grammatrix(jsonfilename):
    cases = from_json(jsonfilename)
    for i,case in enumerate(cases):
        print("\n \033[1m ------ NEXT CASE --------------\033[0m\n")
        print("the gram matrix:\n", matrix(case["G"]), "\n")
        for point in cube_points(case["lincomb_cube"]):
            if point == closest_point_in_cube(case["lincomb_cube"], case["lincomb_LLL"]):
                print('\033[1m' + "THIS IS THE CLOSEST POINT IN THIS CASE" + '\033[0m')
            if point == evaluate_norms_at_cube(cube_points(case["lincomb_cube"]), case["B"])[0][1]:
                print('\033[1m' + "THIS IS THE MINIMAL POINT IN THIS CASE" + '\033[0m')    
            point_versus_grammatrix(case["G"], case["lincomb_LLL"], point)
        if i > 10:
            break

In [5]:
# cube_versus_grammatrix("matrices3x3-600instances.json")

In [6]:
### closest_versus_minimal

def wholify_floatInListAtGivenIndex(inlist, i):
    """
    Given a list, with a float on the i-th position, 
    """
    outlist = deepcopy(inlist)
    for _ in range(len(outlist)):
        vector = outlist.pop(0)
        vector[i] = math.floor(vector[i])
        outlist.append(deepcopy(vector))
        vector[i] = vector[i] + 1
        outlist.append(vector)
    return outlist

def cube_points(lc_cube) -> list:
    """
    Returns a list of linear combinations/vertices of the cube.
    """
    points = [lc_cube]
    for i in range((len(lc_cube))):
        if points[0][i] % 1 != 0: # if current number is not whole 
            points = wholify_floatInListAtGivenIndex(points, i) 
    points = [list(map(lambda x : int(x), point)) for point in points]
    return points
    
def evaluate_norms_at_cube(vertices, B):
    """
    Given an integer cube and returns, the norms of its vertices and corresponding vertices. 
    """
    norms = []
    for vertice in vertices:
        norms.append([(matrix(vertice) * matrix(B)).norm().n(), vertice])
    return sorted(norms)
    
def closest_point_in_cube(lc_cube, lc_LLL) -> list:
    """
    Returns the vertice of the cube closest to the lincomb given by LLL.
    """
    lc_closest = []
    for i in range(len(lc_cube)):
        if lc_LLL[i] > lc_cube[i]:
            lc_closest.append(math.ceil(lc_cube[i]))
        else:
            lc_closest.append(math.floor(lc_cube[i]))
    return lc_closest


def closest_versus_minimal(jsonfilename) -> list:
    "Returns a list of a form [int, int]. The first number indicates  whether the closest vector was the one with shortest norm."
    cases = from_json(jsonfilename)
    stats = [0, 0]
    for case in cases:
        B, lc_cube, lc_LLL = case["B"], case["lincomb_cube"], case["lincomb_LLL"]
        minimal = evaluate_norms_at_cube(cube_points(lc_cube), B)[0][1]
        closest = closest_point_in_cube(lc_cube, lc_LLL)
        if minimal == closest:
            stats[0] += 1
        else:
            stats[1] += 1
    return stats

def closest_versus_minimal_withComments(jsonfilename) -> list:
    "Returns a list of a form [int, int]. The first number indicates  whether the closest vector was the one with shortest norm."
    cases = from_json(jsonfilename)
    stats = [0, 0]
    for case in cases:
        B, lc_cube, lc_LLL = case["B"], case["lincomb_cube"], list(map(lambda x : int(x), case["lincomb_LLL"] ))
        print("lc_LLL: ", list(map(lambda x : int(x), lc_LLL )))
        minimal = evaluate_norms_at_cube(cube_points(lc_cube), B)[0][1] # just a linear combination!
        print("minimal:", minimal)
        closest = closest_point_in_cube(lc_cube, lc_LLL)
        print("closest:", closest)
        print("LLL-min:", [lc_LLL[i] - minimal[i] for i in range(len(minimal))])
        print("LLL-clo:", [lc_LLL[i] - closest[i] for i in range(len(minimal))])
        print(minimal == closest)
        # print(cube_points(lc_cube))
        print()
        if minimal == closest:
            stats[0] += 1
        else:
            stats[1] += 1
    return stats

In [8]:
closest_versus_minimal_withComments("matrices6x6-600instances.json")

lc_LLL:  [-1, -4, -2, 1, 3, 2]


/usr/lib/python3/dist-packages/scipy/sparse/sputils.py:16: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/lib/python3/dist-packages/scipy/linalg/__init__.py:217: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func


minimal: [-1, 0, -1, 0, 0, 0]
closest: [-1, -1, -2, 1, 1, 1]
LLL-min: [0, -4, -1, 1, 3, 2]
LLL-clo: [0, -3, 0, 0, 2, 1]
False

lc_LLL:  [-2, -1, 0, -2, 2, 1]
minimal: [0, -1, 1, 1, 1, 0]
closest: [-1, -1, 0, 0, 2, 1]
LLL-min: [-2, 0, -1, -3, 1, 1]
LLL-clo: [-1, 0, 0, -2, 0, 0]
False

lc_LLL:  [1, 0, -1, -1, 2, -2]
minimal: [1, 0, -1, 0, 1, -1]
closest: [1, 0, -1, -1, 1, -1]
LLL-min: [0, 0, 0, -1, 1, -1]
LLL-clo: [0, 0, 0, 0, 1, -1]
False

lc_LLL:  [8, 6, -2, 7, -11, -4]
minimal: [9, 7, -2, 8, -12, -4]
closest: [8, 7, -2, 8, -12, -4]
LLL-min: [-1, -1, 0, -1, 1, 0]
LLL-clo: [0, -1, 0, -1, 1, 0]
False

lc_LLL:  [0, 0, 0, -1, 1, 0]
minimal: [-1, -2, 0, -2, 1, 1]
closest: [0, -1, 0, -1, 1, 0]
LLL-min: [1, 2, 0, 1, 0, -1]
LLL-clo: [0, 1, 0, 0, 0, 0]
False

lc_LLL:  [4, -1, 1, 7, 0, 1]
minimal: [3, -1, 1, 5, 0, 1]
closest: [3, -1, 1, 5, 0, 1]
LLL-min: [1, 0, 0, 2, 0, 0]
LLL-clo: [1, 0, 0, 2, 0, 0]
True

lc_LLL:  [0, 1, -33, -4, -11, -7]
minimal: [0, 1, -37, -4, -13, -8]
closest: [0, 1, -36, -

[118, 479]